In [1]:
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder,VectorAssembler, StringIndexer, IndexToString, MinMaxScaler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline, Model
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


from pyspark.ml.evaluation import BinaryClassificationEvaluator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder. \
appName("Major Project").\
master("local[1]").\
config("spark.memory.offHeap.enabled","true"). \
config("spark.memory.offHeap.size","10g").getOrCreate()
spark




# Regression 

In [3]:
df=spark.read.csv(r"C:/Users/abhi/Desktop/project feb2023/major project code/small dataset/train2.csv",inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- bidirectional_duration_ms: integer (nullable = true)
 |-- bidirectional_packets: integer (nullable = true)
 |-- bidirectional_bytes: integer (nullable = true)
 |-- src2dst_first_seen_ms: long (nullable = true)
 |-- src2dst_last_seen_ms: long (nullable = true)
 |-- src2dst_packets: integer (nullable = true)
 |-- src2dst_bytes: integer (nullable = true)
 |-- dst2src_first_seen_ms: long (nullable = true)
 |-- dst2src_last_seen_ms: long (nullable = true)
 |-- dst2src_packets: integer (nullable = true)
 |-- dst2src_bytes: integer (nullable = true)
 |-- dst2src_duration_ms: integer (nullable = true)
 |-- src2dst_duration_ms: integer (nullable = true)
 |-- Target: integer (nullable = true)



In [5]:
df.groupBy('Target').count().show()

+------+-------+
|Target|  count|
+------+-------+
|     1|6637795|
|     0|    555|
+------+-------+



<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">?Imbalanced dataset </p>

1.Oversampling   
2.Undersampling    
4.By class weight     
3.For Random forest estimator we can give threshold value in parameter to handle

In [6]:
# code block 1
from pyspark.sql.functions import col, explode, array, lit

#df.show()
major_df = df.filter(col("Target") == 1)
minor_df = df.filter(col("Target") == 0)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 11959


<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">?Imbalanced dataset is  ratio: 11959</p>

In [7]:
a = range(ratio)
# duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')
# combine both oversampled minority rows and previous majority rows 
combined_df = major_df.unionAll(oversampled_df)
combined_df.limit(5).toPandas()

,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,src2dst_last_seen_ms,src2dst_packets,src2dst_bytes,dst2src_first_seen_ms,dst2src_last_seen_ms,dst2src_packets,dst2src_bytes,dst2src_duration_ms,src2dst_duration_ms,Target
0,0,1,110,1532103708659,1532103708659,1,110,0,0,0,0,0,0,1
1,0,1,110,1532103730254,1532103730254,1,110,0,0,0,0,0,0,1
2,0,1,110,1532106434466,1532106434466,1,110,0,0,0,0,0,0,1
3,0,1,110,1532106807748,1532106807748,1,110,0,0,0,0,0,0,1
4,0,1,110,1532106876197,1532106876197,1,110,0,0,0,0,0,0,1


In [8]:
combined_df.groupBy('Target').count().show()

+------+-------+
|Target|  count|
+------+-------+
|     1|6637795|
|     0|6637245|
+------+-------+



In [9]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)


combined_df_2.limit(5).toPandas()


,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,src2dst_last_seen_ms,src2dst_packets,src2dst_bytes,dst2src_first_seen_ms,dst2src_last_seen_ms,dst2src_packets,dst2src_bytes,dst2src_duration_ms,src2dst_duration_ms,Target
0,0,1,60,1532101437211,1532101437211,1,60,0,0,0,0,0,0,1
1,0,1,60,1532102392466,1532102392466,1,60,0,0,0,0,0,0,1
2,0,1,60,1532104050367,1532104050367,1,60,0,0,0,0,0,0,1
3,0,1,60,1532104245844,1532104245844,1,60,0,0,0,0,0,0,1
4,0,1,60,1532104611765,1532104611765,1,60,0,0,0,0,0,0,1


In [10]:
combined_df_2.groupBy('Target').count().show()

+------+-----+
|Target|count|
+------+-----+
|     1|  557|
|     0|  555|
+------+-----+



In [15]:
dataset_size=float(df.select("Target").count())

numPositives=df.select("Target").where('Target == 1').count()
numNegatives=float(dataset_size-numPositives)

per_ones=(float(numPositives)/float(dataset_size))*100
per_zero=(float(numNegatives)/float(dataset_size))*100

print('The number of 1 {}'.format(numPositives))
print('The number of 0 are {}'.format(numNegatives))

print('Percentage of ones are {}'.format(per_ones))
print('Percentage of zero are {}'.format(per_zero))


The number of 1 6637795
The number of 0 are 555.0
Percentage of ones are 99.99163948872838
Percentage of zero are 0.008360511271626231


In [12]:
BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))


BalancingRatio= numPositives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 8.360511271626232e-05
BalancingRatio = 0.9999163948872838


In [13]:
from pyspark.sql.functions import *
df=df.withColumn("classWeights",when(df.Target == 0,BalancingRatio).otherwise(1-BalancingRatio))
df.select("classWeights").show(5)

+------------------+
|      classWeights|
+------------------+
|0.9999163948872838|
|0.9999163948872838|
|0.9999163948872838|
|0.9999163948872838|
|0.9999163948872838|
+------------------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import *
df=df.withColumn("classWeights",when(df.Target == 1,BalancingRatio).otherwise(1-BalancingRatio))
df.select("classWeights").show(5)

+--------------------+
|        classWeights|
+--------------------+
|8.360511271621185E-5|
|8.360511271621185E-5|
|8.360511271621185E-5|
|8.360511271621185E-5|
|8.360511271621185E-5|
+--------------------+
only showing top 5 rows

